In [1]:
import pandas as pd
import numpy as np
import os

from folium.features import DivIcon
import folium
import pyproj
from pyproj import Proj, transform

%matplotlib inline  
import matplotlib.pyplot as plt 
import seaborn as sns 

import warnings
warnings.filterwarnings(action='ignore')

def katec_to_wgs84(x, y):
    inProj  = Proj('+proj=tmerc +lat_0=38 +lon_0=128 +k=0.9999 +x_0=400000 +y_0=600000 +ellps=bessel +units=m +no_defs +towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43')
    outProj = Proj({ 'proj':'latlong', 'datum':'WGS84', 'ellps':'WGS84' })
    return transform( inProj, outProj, x, y )

WGS84 = { 'proj':'latlong', 'datum':'WGS84', 'ellps':'WGS84', }
KATEC = { 'proj':'tmerc', 'lat_0':'38N', 'lon_0':'128E', 'ellps':'bessel','x_0':'400000', 'y_0':'600000', 'k':'0.9999','towgs84':'-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43'}
def wgs84_to_katec(longitude, latitude):
    return transform( Proj(**WGS84), Proj(**KATEC), longitude, latitude )

##### 현재 작업 위치 확인


In [2]:
pwd

'C:\\Users\\dudtj\\busan_moving_person\\custom_code'

In [3]:
cd ../data/store_data/100test/

C:\Users\dudtj\busan_moving_person\data\store_data\100test


In [45]:
geodf['total'].sort_values(ascending=False)[:25].values

array([984, 289, 249, 241, 221, 158, 155, 120, 118,  92,  87,  84,  77,
        74,  72,  71,  71,  69,  69,  67,  67,  66,  65,  63,  61],
      dtype=int64)

In [50]:
kcelldf = pd.read_csv('CV100_NEW_FPR_BUSANJINGU_STORE_2021.csv', encoding='UTF-8') # 이름변경필요부분
tdf = pd.DataFrame(kcelldf['total'].groupby(kcelldf['id']).sum()).reset_index()
geodf = pd.merge(tdf, kcelldf[['id', 'x', 'y']] , how = 'left', on = 'id')[['id', 'x', 'y', 'total']].drop_duplicates()
geodf.to_csv('./100c_busan_store_dense.csv', index=False, encoding='UTF-8') # 이름변경필요부분
print("making success :)")

def f_color(x):
    if x in geodf['total'].sort_values(ascending=False)[:50].values: # 60
        a = '#FF4F00'
    elif x in geodf['total'].sort_values(ascending=False)[50:100].values:      # 75%를 기준으로 설정
        a = '#FCB100'   
    elif x in geodf['total'].sort_values(ascending=False)[100:150].values:      # 50%를 기준으로 설정
        a = '#E0F500' 
    elif x in geodf['total'].sort_values(ascending=False)[150:200].values:      # 25%를 기준으로 설정
        a = '#8CF700'  
    else:
        a = '#ffffff' #'#00F000' 
    return a

making success :)


In [51]:
geodf['cell_color']= geodf['total'].apply(lambda x : f_color(x))

print("function success :)")

###################################################################
### 셀 수 변경###
cell_size = 100 #
#################
a = cell_size//2
x = geodf
x['nwx'], x['nwy'] = x['x']-a, x['y']+a
x['nex'], x['ney'] = x['x']+a, x['y']+a
x['swx'], x['swy'] = x['x']-a, x['y']-a
x['sex'], x['sey'] = x['x']+a ,x['y']-a

x['lng'], x['lat'] = katec_to_wgs84(x.x.to_list(), x.y.to_list())
x['nwlng'], x['nwlat'] = katec_to_wgs84(x.nwx.to_list(), x.nwy.to_list())
x['nelng'], x['nelat'] = katec_to_wgs84(x.nex.to_list(), x.ney.to_list())
x['swlng'], x['swlat'] = katec_to_wgs84(x.swx.to_list(), x.swy.to_list())
x['selng'], x['selat'] = katec_to_wgs84(x.sex.to_list(), x.sey.to_list())

map = folium.Map(location=[geodf['lat'].mean(), geodf['lng'].mean()], zoom_start=13, tiles='stamentoner')

for n in x.index:
    nw, ne, sw, se = [x.loc[n, 'nwlat'],x.loc[n, 'nwlng']],[x.loc[n, 'nelat'],x.loc[n, 'nelng']],[x.loc[n, 'swlat'],x.loc[n, 'swlng']],[x.loc[n, 'selat'],x.loc[n, 'selng']]
    cell_color = x.loc[n, 'cell_color']
    loc_name = 'ID:'+str(round(x.loc[n, 'id'],3)) + '       Total:'+ str(round(x.loc[n, 'total'],3))
    folium.Polygon(locations=[nw, sw, se, ne],
                   color='white',
                   popup=loc_name,
                   weight=0.55,
                   fill=True,
                   fill_color=cell_color,
                   fill_opacity=0.6
     ).add_to(map)

map.save('./top200_busan_store_dense.html') ###### 이름변경필요부분
# map
print("done :)")
map

function success :)
done :)


In [54]:
print(geodf['total'].sort_values(ascending=False)[:50].values.min()) # 60
print(geodf['total'].sort_values(ascending=False)[50:100].values.min()) # 36
print(geodf['total'].sort_values(ascending=False)[100:150].values.min()) # 28
print(geodf['total'].sort_values(ascending=False)[150:200].values.min()) # 24

49
36
28
24


In [56]:
print(geodf['total'].sort_values(ascending=False)[:50].values.max())# 60
print(geodf['total'].sort_values(ascending=False)[50:100].values.max()) # 36
print(geodf['total'].sort_values(ascending=False)[100:150].values.max()) # 28
print(geodf['total'].sort_values(ascending=False)[150:200].values.max()) # 24

984
48
35
27
